<a href="https://colab.research.google.com/github/itz-abhay/Hospital-Cost-Prediction/blob/main/Hospital_cost_prediction_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Verify A



In [ ]:
import numpy as np
from termcolor import colored


def verify_part_a():
    """
    Verifying results of Part a.
    :return: Nothing
    """

    # Get the predicted and actual weights, output files.
    actual_output = np.loadtxt("/content/model_outputfile_a.txt")
    actual_weights = np.loadtxt("/content/model_weightfile_a.txt")
    predicted_output = np.loadtxt("/content/Part_a/predictions.txt")
    predicted_weights = np.loadtxt("/content/Part_a/weights.txt")

    if (actual_output.shape[0] != predicted_output.shape[0]):
        print(colored("Prediction file of wrong dimensions for part a", "red"))
        exit()
    if (actual_weights.shape[0] != predicted_weights.shape[0]):
        print(colored("Weight file of wrong dimensions for part a", "red"))
        exit()

    pred_error = np.sum(np.square(predicted_output - actual_output)) / np.sum(
        np.square(actual_output))  # Error in output
    weight_error = np.sum(np.square(predicted_weights - actual_weights)) / np.sum(
        np.square(actual_weights))  # Error in weights

    print(colored("Error in Predictions for part a : " + str(pred_error), "green"))
    print(colored("Error in Weights for part a : " + str(weight_error), "green"))

#Verify B

In [ ]:
import numpy as np
from termcolor import colored


def verify_part_b():
    """
    Verifying results of Part b.
    :return: Nothing
    """

    # Get the predicted and actual weights, output files.
    actual_output = np.loadtxt("/content/model_outputfile_b.txt")
    actual_weights = np.loadtxt("/content/model_weightfile_b.txt")
    predicted_output = np.loadtxt("/content/Part_b/predictions.txt")
    predicted_weights = np.loadtxt("/content/Part_b/weights.txt")

    if (actual_output.shape[0] != predicted_output.shape[0]):
        print(colored("Prediction file of wrong dimensions for part b", "red"))
        exit()
    if (actual_weights.shape[0] != predicted_weights.shape[0]):
        print(colored("Weight file of wrong dimensions for part b", "red"))
        exit()

    pred_error = np.sum(np.square(predicted_output - actual_output)) / np.sum(
        np.square(actual_output))  # Error in output
    weight_error = np.sum(np.square(predicted_weights - actual_weights)) / np.sum(
        np.square(actual_weights))  # Error in weights

    print(colored("Error in Predictions for part b : " + str(pred_error), "green"))
    print(colored("Error in Weights for part b : " + str(weight_error), "green"))

#Cross Validation

In [ ]:
import numpy as np
from sklearn.metrics import r2_score


def KFoldCrossValidation(X_train, Y_train, model, k=10):
    """
    Do k-fold cross validation using the model. The score function of k-fold CV is R2 Score.
    :param X_train: Training data
    :param Y_train: Test data
    :param model: model to be used for learning weights of features
    :param k: k of k-fold cross CV
    :return: k-fold CV R2 Score
    """
    r = int(X_train.shape[0] / k)
    k_fold_r2_score = 0.0
    fold = 1
    while (fold <= k):
        X1 = X_train[0:(fold - 1) * r]
        Y1 = Y_train[0:(fold - 1) * r]
        X2 = X_train[(fold - 1) * r:fold * r]
        Y2 = Y_train[(fold - 1) * r:fold * r]
        X3 = X_train[fold * r:(fold + 1) * r]
        Y3 = Y_train[fold * r:(fold + 1) * r]

        X_tra = np.concatenate((X1, X3), axis=0)
        Y_tra = np.concatenate((Y1, Y3), axis=0)

        model.fit(X_tra, Y_tra)
        predictions = model.predict(X2)
        k_fold_r2_score += r2_score(Y2, predictions)
        fold += 1

    k_fold_r2_score /= k
    return k_fold_r2_score

#Part A

In [ ]:
import os

import numpy as np

from sklearn.linear_model import LinearRegression


def solve_a(X_train, Y_train, X_test, results_path):
    """
    Do Part a i.e. use linear regression to predict total costs.
    :param X_train: Training data
    :param Y_train: Training labels
    :param X_test: Test data
    :param results_path: Path to store the results
    :return: Nothing
    """

    model = LinearRegression()
    model.fit(X_train,Y_train)
    prediction = model.predict(X_test)
    weights = model.coef_  # weights of the features

    results_path = os.path.join(results_path, "Part_a")
    try:
        os.mkdir(results_path)
    except:
        do_nothing = True

    np.savetxt(os.path.join(results_path, "weights.txt"), weights)
    np.savetxt(os.path.join(results_path, 'predictions.txt'), prediction)

#Part B

In [ ]:
import os

import numpy as np
import pandas as pd

# from src.CrossValidation import KFoldCrossValidation
from sklearn.linear_model import Ridge


def solve_b(X_train, Y_train, X_test, results_path, regc, k):
    """
    Do Part b i.e. use ridge regression to predict total costs.
    :param X_train: Training data
    :param Y_train: Training labels
    :param X_test: Test data
    :param results_path: Path to save results
    :param regc: List storing the regression penalties which will be used in ridge regression
    :param k: k of k-fold CV
    :return: Nothing
    """

    bp = regc[0]  # Best Regression Penalty
    best_r2_score = 0  # Best k-fold R2 Score

    df_result = pd.DataFrame(columns=['Ridge Penalty', str(k) + ' - fold R2 Score'])
    for l in regc:
        model = Ridge(alpha=l)
        score = KFoldCrossValidation(X_train=X_train, Y_train=Y_train, model=model, k=k)

        dic = {'Ridge Penalty': l, str(k) + ' - fold R2 Score': score}
        df_result = df_result.append(dic, ignore_index=True)

        if score > best_r2_score:
            best_r2_score = score
            bp = l

    model = Ridge(alpha=bp)  # Best Ridge Regression model
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)
    weights = model.coef_  # Weights of features

    results_path = os.path.join(results_path, "Part_b")
    try:
        os.mkdir(results_path)
    except:
        do_nothing = True

    np.savetxt(os.path.join(results_path, "weights.txt"), weights)
    np.savetxt(os.path.join(results_path, "predictions.txt"), predictions)
    df_result.to_csv(os.path.join(results_path, "results_ridge_penalty.csv"), index=False)

#Feature Enginnering

In [ ]:
import numpy as np


def onehotencoding(data, feature_name, target_feature, high_freq_features):
    """
    Do one hot encoding of the feature = feature_name
    :param data: data including both train and test data
    :param feature_name: name of the feature to be one hot encoded
    :param target_feature: target feature to be used in one-hot encoding
    :param high_freq_features: features having high unique values
    :return: one hot encoded data
    """
    unique_list = data[feature_name].value_counts(sort=True, ascending=False).index
    n_arr = np.asarray(data[feature_name].values)
    target_arr = np.asarray(data[target_feature].values)
    for idx, value in enumerate(unique_list):
        data[feature_name + '_' + str(idx)] = np.where(n_arr == value, target_arr, 0)
        if idx >= 20 and feature_name in high_freq_features:
            # Get one hot encoding of best 20 values only
            break
    return data


def multifeature_onehotencoding(data, feature_name1, feature_name2, target_feature):
    """
    Do multi feature one hot encoding of 2 features feature_name1 and feature_name2
    :param data: Data including both training and test data
    :param feature_name1: First feature to be used in one hot encoding
    :param feature_name2: Second feature to be used in one hot encoding
    :param target_feature: target feature to be used in one hot encoding
    :return: one hot encoded data
    """

    unique_list1 = data[feature_name1].value_counts(sort=True, ascending=False).index
    n_arr1 = np.asarray(data[feature_name1].values)
    unique_list2 = data[feature_name2].value_counts(sort=True, ascending=False).index
    n_arr2 = np.asarray(data[feature_name2].values)
    target_arr = np.asarray(data[target_feature].values)
    cnt = 0
    for idx1, value1 in enumerate(unique_list1):
        for idx2, value2 in enumerate(unique_list2):
            data[feature_name1 + '_' + feature_name2 + '_' + str(cnt)] = np.where(
                (n_arr1 == value1) & ((n_arr2 == value2)), target_arr, 0)
            cnt += 1
    return data


def feature_engineering(data):
    """
    Do feature engineering on the data.
    :param data: DataFrame storing training and test data
    :return: data after feature engineering
    """

    # Features to one hot encode
    features_to_one_hot_encode = ['Ethnicity', 'Type of Admission', 'Age Group', 'Operating Certificate Number',
                                  'Payment Typology 1', 'APR Severity of Illness Description',
                                  'APR Risk of Mortality', 'APR Medical Surgical Description',
                                  'APR MDC Description', 'CCS Procedure Description',
                                  'CCS Diagnosis Description', 'APR DRG Description']

    # High Frequency features
    high_frequency_features = ['CCS Procedure Description', 'CCS Diagnosis Description', 'APR DRG Description']

    for feature_name in features_to_one_hot_encode:
        data = onehotencoding(data=data, feature_name=feature_name, target_feature='Length of Stay',
                              high_freq_features=high_frequency_features)

    # Features to multi one hot encode
    multi_features_to_one_hot_encode = [('Health Service Area', 'Emergency Department Indicator'),
                                        ('Age Group', 'Emergency Department Indicator')]

    for feature_name1, feature_name2 in multi_features_to_one_hot_encode:
        data = multifeature_onehotencoding(data, feature_name1, feature_name2, target_feature='Length of Stay')

    # Features to drop
    features_to_drop = ['Ethnicity', 'Type of Admission', 'Age Group', 'Operating Certificate Number',
                        'Payment Typology 1', 'APR Severity of Illness Description',
                        'APR Risk of Mortality', 'APR Medical Surgical Description',
                        'Race', 'Gender', 'Facility Id', 'CCS Diagnosis Code', 'CCS Procedure Code', 'APR DRG Code',
                        'APR MDC Code']

    data.drop(features_to_drop, inplace=True, axis=1)

    return data

#Part C

In [ ]:
import math
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from termcolor import colored
from sklearn.linear_model import Lasso, Ridge

# from src.CrossValidation import KFoldCrossValidation
# from src.feature_engineering import feature_engineering
# from src.models.LassoRegression import LassoRegression
# from src.models.RidgeRegression import RidgeRegression

plt.rcParams['figure.figsize'] = [10, 10]


def plots(predictions, actual, results_path, LOS):
    """
    Make plots
    :param predictions: Predicted values
    :param actual: Actual values
    :param results_path: Path to store the results
    :param LOS: Length of Stay training data
    :return: Nothing
    """

    residuals = predictions - actual

    results_path = os.path.join(results_path, 'Plots')
    try:
        os.mkdir(results_path)
    except:
        do_nothing = True

    # Create Total Costs vs LOS plot
    plt.scatter(LOS, actual, s=1)
    plt.title('Total Costs vs Length of Stay')
    plt.xlabel('Length of Stay')
    plt.ylabel('Total Costs')
    plt.savefig(os.path.join(results_path, 'TotalCostsVsLOS.png'))
    plt.clf()

    # Create Predicted Total Costs vs Actual Total Costs plot
    val = np.polyfit(x=actual, y=predictions, deg=1)
    m = val[0]
    b = val[1]
    plt.scatter(actual, predictions, s=1)
    plt.plot(actual, m * actual + b, c='k')  # Best fit line
    plt.title('Predictions vs Actual')
    plt.xlabel('Actual')
    plt.ylabel('Predictions')
    plt.savefig(os.path.join(results_path, 'PredictionsVsActual.png'))
    plt.clf()

    # Create residuals vs Actual Total Costs plot
    plt.scatter(actual, residuals, s=1)
    plt.title('Residuals vs Actual')
    plt.xlabel('Actual')
    plt.ylabel('Residuals')
    plt.savefig(os.path.join(results_path, 'ResidualsVsActual.png'))
    plt.clf()

    # Create histogram plot of residuals
    plt.hist(residuals, bins=1000, range=(-1e5, 1e5))
    plt.title('Density Plot of Residuals')
    plt.ylabel('Count')
    plt.xlabel('Residuals')
    plt.savefig(os.path.join(results_path, 'Histogram.png'))
    plt.clf()


def use_correlation(X_train, Y_train, X_test):
    """
    Find the best power p of a feature so that power(feature,p) and Y_train have high absolute value of correlation.
    :param X_train: Training data
    :param Y_train: Training labels
    :param X_test: Test data
    :return: X_train and X_test with updated values
    """

    for idx in range(X_train.shape[1]):
        p = 0.1
        max_corr = 0
        best_p = 0.1
        while (p <= 1.5):
            corr = abs(np.corrcoef(Y_train, np.power(X_train[:, idx], p))[0][1])
            if corr > max_corr:
                max_corr = corr
                best_p = p
            p += 0.1
        X_train[:, idx] = np.power(X_train[:, idx], best_p)
        X_test[:, idx] = np.power(X_test[:, idx], best_p)

    return X_train, X_test


def solve_c(train_data, test_data, results_path, k, get_features_importance, reg_lower_limit, reg_upper_limit,
            random_searches):
    """
    Do part c i.e. use feature engineering to improve model performance.
    :param train_data: Training data
    :param test_data: Test data
    :param results_path: Path to store the results
    :param k: k of k-fold CV
    :param get_important_features: Whether to get important features or not using Lasso Regression
    :param reg_lower_limit: Lower limit of regularisation penalty
    :param reg_upper_limit: Upper limit of regularisation penalty
    :param random_searches: Number of random searches in [reg_lower_limit,reg_upper_limit] to find the
                            best regularisation penalty
    :return: Nothing
    """

    X_train = train_data.iloc[:, 1:-1]
    Y_train = np.asarray(train_data['Total Costs'].values)
    X_test = test_data.iloc[:, 1:-1]

    col = np.ones(X_train.shape[0])
    X_train['bias'] = col
    col = np.ones(X_test.shape[0])
    X_test['bias'] = col

    model = Ridge(alpha=0.001)
    without_feature_engineering_k_fold_r2_score = KFoldCrossValidation(X_train=np.asarray(X_train),
                                                                       Y_train=Y_train, model=model, k=k)

    print(colored(str(k) + '-Fold R2 Score (Without Feature Engineering) : ' +
                  str(without_feature_engineering_k_fold_r2_score), 'cyan'))

    print(colored('Feature Engineering started ...', 'cyan'))
    # Remove outliers
    non_outliers = (Y_train <= 2e5)
    Y_train = Y_train[non_outliers]
    X_train = X_train.loc[non_outliers]

    data = pd.concat([X_train, X_test], axis=0)

    # Feature Engineering
    data = feature_engineering(data=data)

    index_bias = list(data.columns).index('bias')  # Index of bias in features list

    X_train = np.asarray(data.iloc[0:X_train.shape[0]])
    X_test = np.asarray(data.iloc[X_train.shape[0]:])

    X_train, X_test = use_correlation(X_train=X_train, Y_train=Y_train, X_test=X_test)

    print(colored('Finished Feature Engineering', 'cyan'))

    print(colored('\nFinding best regularisation penalty started ...', 'cyan'))

    best_r2_score = 0  # Best k-fold R2 score
    bp = 0  # Best Ridge Regression Penalty
    df_result = pd.DataFrame(columns=['Ridge Penalty', str(k) + ' - fold R2 Score'])
    for idx in range(random_searches):
        l = random.uniform(reg_lower_limit, reg_upper_limit)
        model = Ridge(alpha=l)
        score = KFoldCrossValidation(X_train=X_train, Y_train=Y_train, model=model, k=k)

        dic = {'Ridge Penalty': l, str(k) + ' - fold R2 Score': score}
        df_result = df_result.append(dic, ignore_index=True)

        if score > best_r2_score:
            best_r2_score = score
            bp = l

    print(colored('Best ' + str(k) + '-Fold R2 Score after feature engineering : ' + str(best_r2_score), 'cyan'))

    improvemet_r2_score = (best_r2_score - without_feature_engineering_k_fold_r2_score) * 100
    improvemet_r2_score /= without_feature_engineering_k_fold_r2_score
    improvemet_r2_score = round(improvemet_r2_score, 2)

    print(colored('Improvement in ' + str(k) + ' - Fold R2 Score due to feature engineering : '
                  + str(improvemet_r2_score) + ' %', 'cyan'))

    model = Ridge(alpha=bp)
    model.fit(X_train, Y_train)

    X_test = pd.DataFrame(X_test)
    X_test.fillna(value=0, inplace=True) #Make sure there are no missing values in X_test
    predictions = model.predict(X_test)
    weights = model.coef_

    results_path = os.path.join(results_path, "Part_c")
    try:
        os.mkdir(results_path)
    except:
        do_nothing = True

    np.savetxt(os.path.join(results_path, "weights.txt"), weights)
    np.savetxt(os.path.join(results_path, "predictions.txt"), predictions)
    df_result.to_csv(os.path.join(results_path, "results_ridge_penalty.csv"), index=False)

    train_predictions = model.predict(X_train)

    plots(predictions=train_predictions, actual=Y_train,
          results_path=results_path,
          LOS=data['Length of Stay'][0:X_train.shape[0]].values)

    if get_features_importance:
        # Use Lasso Regression to get feature importance
        print(colored('Calculating Importance of features ...', 'cyan'))
        model = Lasso(alpha=bp)
        model.fit(X_train, Y_train)
        weights = model.coef_
        features_importance = []
        correlation = []
        feature_names = data.columns
        for idx, coef in enumerate(weights):
            importance = abs(coef) * np.mean(X_train[:, idx])  # Importance of that feature
            features_importance.append((feature_names[idx], importance))

        for idx in range(X_train.shape[1]):
            corr = abs(np.corrcoef(Y_train, X_train[:, idx])[0][1])
            if math.isnan(corr) is False:
                correlation.append((feature_names[idx], corr))

        features_importance.sort(key=lambda x: x[1], reverse=True)
        correlation.sort(key=lambda x: x[1], reverse=True)

        with open(os.path.join(results_path, "features_importance.txt"), 'w') as f:
            f.write('Feature importance from highest to lowest : ' + '\n')
            f.write('\n')
            idx = 1
            for feature, importance in features_importance:
                f.write(str(idx) + '. ' + feature + ' : ' + str(importance) + '\n')
                idx += 1

        with open(os.path.join(results_path, "correlation.txt"), 'w') as f:
            f.write('Correlation between features and Total Costs from highest to lowest : ' + '\n')
            f.write('\n')
            idx = 1
            for feature, corr in correlation:
                f.write(str(idx) + '. ' + feature + ' : ' + str(corr) + '\n')
                idx += 1

        print(colored('Importance of features calculated', 'cyan'))

#Run

In [ ]:
import os
import time
import warnings

import numpy as np
import pandas as pd
import yaml
from termcolor import colored


warnings.filterwarnings("ignore")

os.system('color')
# Read Parameters.yml file
with open("Parameters.yml", 'r') as stream:
    dic = yaml.safe_load(stream)

# Get training and test data
train_data = pd.read_csv(dic['train_data'])
test_data = pd.read_csv(dic['test_data'])

# Total Costs is the last column of training and test data
Y_train = np.asarray(train_data['Total Costs'])
X_train = np.asarray(train_data.iloc[:, 1:-1])
X_test = np.asarray(test_data.iloc[:, 1:])

# Adding column of ones to handle the bias
X_train = np.c_[np.ones(X_train.shape[0]), X_train]
X_test = np.c_[np.ones(X_test.shape[0]), X_test]

regc = dic['regularisation_penalty']
results_path = dic['results_path']
k = dic['k']
get_features_importance = dic['get_features_importance']
verify_res = dic['verify_results']
reg_lower_limit = dic['regularisation_penalty_lower_limit']
reg_upper_limit = dic['regularisation_penalty_upper_limit']
random_searches = dic['random_searches']

if "a" in dic['parts']:
    # Do Part a : Use Linear Regression to predict Total Costs
    start = time.time()
    print(colored('Part a started ... ', 'magenta'))
    solve_a(X_train=X_train, Y_train=Y_train, X_test=X_test, results_path=results_path)
    if verify_res:
        verify_part_a()
    end = time.time()
    print(colored('Part a finished (' + str(round(end - start, 2)) + ' s)', 'magenta'))

if "b" in dic['parts']:
    # Do Part b : Use Ridge Regression to predict Total Costs
    start = time.time()
    print(colored('\n\nPart b started ... ', 'magenta'))
    solve_b(X_train=X_train, Y_train=Y_train, X_test=X_test, results_path=results_path, regc=regc, k=k)
    if verify_res:
        verify_part_b()
    end = time.time()
    print(colored('Part b finished (' + str(round(end - start, 2)) + ' s)', 'magenta'))

if "c" in dic['parts']:
    # Do Part c : Use Feature Engineering to improve R2 Score
    start = time.time()
    print(colored('\n\nPart c started ... ', 'magenta'))
    solve_c(train_data=train_data, test_data=test_data, results_path=results_path, k=k,
            get_features_importance=get_features_importance, reg_lower_limit=reg_lower_limit,
            reg_upper_limit=reg_upper_limit, random_searches=random_searches)
    end = time.time()
    print(colored('Part c finished (' + str(round(end - start, 2)) + ' s)', 'magenta'))

Part a started ... 
Error in Predictions for part a : 2.0414601545564422e-23
Error in Weights for part a : 0.09146520729281783
Part a finished (3.15 s)


Part b started ... 
Error in Predictions for part b : 5.910963915400773e-17
Error in Weights for part b : 0.09146533372437309
Part b finished (18.06 s)


Part c started ... 
10-Fold R2 Score (Without Feature Engineering) : 0.5607403445822712
Feature Engineering started ...
Finished Feature Engineering

Finding best regularisation penalty started ...
Best 10-Fold R2 Score after feature engineering : 0.7780019182825523
Improvement in 10 - Fold R2 Score due to feature engineering : 38.75 %
Calculating Importance of features ...
Importance of features calculated
Part c finished (689.61 s)


<Figure size 1000x1000 with 0 Axes>